In [31]:
# Process images on birds using whole training set (no human refinement)
# Using Dense Neural Network

from tensorflow.data import Dataset 
from tensorflow.image import decode_image
from tensorflow import read_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os

In [5]:
# First load the data set

PATH = os.getcwd()

# The ImageDataGenerator object assumes that resources will be preclassified, i.e., 
# resources/A will contain all images of "A" type
# resources/B will contain all images of "B" type
# The object "classifies" data based on the directory
training_data_dir = os.path.join(PATH, 'scrape', 'resources')

training_data_image_dir = os.path.join(training_data_dir, 'birds')

training_data_size = len(os.listdir(training_data_image_dir))


In [6]:
print(training_data_image_dir)

/home/franklin/Documents/dev/birdee/scrape/resources/birds


In [7]:
print(training_data_size)

998


In [8]:
# what are the sizes of the photographs?

from PIL import Image

sizes = {}
for img in os.listdir(training_data_image_dir):
    img_path = os.path.join(training_data_image_dir, img)
    (width, height) = Image.open(img_path).size
    if (width, height) in sizes:
        sizes[(width, height)] = sizes[(width, height)] + 1
    else:
        sizes[(width, height)] = 1
        
print(sizes)

{(800, 533): 478, (896, 597): 322, (533, 800): 65, (1000, 771): 1, (1137, 794): 1, (969, 1000): 1, (467, 700): 2, (700, 467): 3, (900, 600): 57, (1000, 667): 3, (599, 800): 1, (597, 896): 35, (600, 900): 3, (800, 800): 4, (800, 556): 1, (800, 538): 1, (800, 400): 4, (1200, 600): 1, (800, 581): 1, (900, 651): 1, (800, 534): 1, (5184, 3456): 1, (800, 623): 1, (4894, 3263): 1, (799, 533): 3, (1000, 702): 1, (514, 800): 1, (900, 465): 1, (900, 795): 1, (1000, 755): 1, (1200, 538): 1}


In [37]:
# most images are in the 800 x 533 size
# let's bump the size to 800 x 600 to be accomodating

IMAGE_WIDTH = 800
IMAGE_HEIGHT = 600


In [42]:
BATCH_SIZE = 64

In [39]:
# Prepare data - read images from disk

train_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                           directory=training_data_dir,
                                                           shuffle=True,
                                                           target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                                           class_mode='binary')

Found 998 images belonging to 1 classes.


In [26]:
# Create the model

model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [32]:
# Compile the model

model.compile(optimizer='adam',
              loss=BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [33]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 600, 800, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 300, 400, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 300, 400, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 200, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 200, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 100, 64)       0         
_________________________________________________________________
flatten (Flatten)            (None, 480000)            0

In [45]:
EPOCHS = 15 

# Train the model

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=training_data_size // BATCH_SIZE,
    epochs=EPOCHS
    # validation_data=val_data_gen,
    # validation_steps=total_val // BATCH_SIZE
)


Epoch 1/15
15/15 [==============================] - 209s 14s/step - loss: 0.0463 - acc: 1.0000
Epoch 2/15
15/15 [==============================] - 183s 12s/step - loss: 0.0000e+00 - acc: 1.0000
Epoch 3/15
15/15 [==============================] - 193s 13s/step - loss: 0.0000e+00 - acc: 1.0000
Epoch 4/15
15/15 [==============================] - 173s 12s/step - loss: 0.0000e+00 - acc: 1.0000
Epoch 5/15
15/15 [==============================] - 179s 12s/step - loss: 0.0000e+00 - acc: 1.0000
Epoch 6/15
15/15 [==============================] - 239s 16s/step - loss: 0.0000e+00 - acc: 1.0000
Epoch 7/15
15/15 [==============================] - 226s 15s/step - loss: 0.0000e+00 - acc: 1.0000
Epoch 8/15
15/15 [==============================] - 244s 16s/step - loss: 0.0000e+00 - acc: 1.0000
Epoch 9/15
15/15 [==============================] - 227s 15s/step - loss: 0.0000e+00 - acc: 1.0000
Epoch 10/15
15/15 [==============================] - 245s 16s/step - loss: 0.0000e+00 - acc: 1.0000
Epoch 11/15
1

In [47]:
acc = history.history

In [48]:
print(acc)

{'loss': [0.047489795235546035, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'acc': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
